In [1]:
import os
import sys
import re
import pandas as pd
import commands
import ast
import itertools
import pyspark.sql.functions
from pyspark.sql.functions import col
from datetime import datetime
from collections import Counter
import re
import numpy as np
#from pyspark.sql import SparkSession
spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
if not os.path.isdir(spark_home):
    raise ValueError('SPARK_HOME environment variable is not a directory')
if not os.path.isdir(os.path.join(spark_home, 'python')):
    raise ValueError('SPARK_HOME directory does not contain python')
sys.path.insert(0, os.path.join(spark_home, 'python'))
pylib_list = (item for item in os.listdir(os.path.join(spark_home, 'python/lib/'))
              if re.match(r'py4j-\d+(\.\d+)+-src\.zip\Z', item)
              )
try:
    py4j_file = max(pylib_list)
    py4j = os.path.join(spark_home, os.path.join('python/lib', py4j_file))
except ValueError:
    raise ValueError(
        'Could not find py4j'
    )
sys.path.insert(0, py4j)

spark_release_file = spark_home + "/RELEASE"
if os.path.exists(spark_release_file) and "Spark" in  open(spark_release_file).read():
    pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", " --master yarn-client \
                                         --executor-memory 4g --executor-cores 5 --driver-memory 16g"
                                        )
    if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
    os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

with open(os.path.join(spark_home, 'python/pyspark/shell.py')) as f:
    code = compile(f.read(), os.path.join(spark_home, 'python/pyspark/shell.py'), 'exec')
    exec(code)


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.0.2.6.5.0-292
      /_/

Using Python version 2.7.15 (default, May  1 2018 23:32:55)
SparkSession available as 'spark'.


# Convert multitab in single tab excel

In [2]:
tab_list=['account','activityparty','annotation','appointment','businessunit','calendar','calendarrule','contact','cxlvhlp_chatactivity','cxlvhlp_chatqueuestatistic','cxlvhlp_surveyitem','email','fax','gcct_accountresponsibleagent','gcct_additionalsymptomcodes','gcct_addportalmessage','gcct_arbitrationclaimprocessing','gcct_buybackevaluationmilestones','gcct_caseassignment','gcct_caseclassification','gcct_casedispositiontype','gcct_coachback','gcct_country','gcct_customersatisfactiontools','gcct_delegationofauthority','gcct_demandltrtpsmclaimsprocessing','gcct_doaprogramcode','gcct_documentcustomerrecontact','gcct_engine','gcct_executive','gcct_executiveliaison','gcct_fieldinvolvementassistancerequest','gcct_fmcccontract','gcct_fulfillmentvendor','gcct_genericinformation','gcct_genericinformationtopic','gcct_generictopic','gcct_goodfaithreview','gcct_knownpartsdelay','gcct_lemonlawcriteria','gcct_loanercost','gcct_loyaltyassistance','gcct_loyaltycostdetails','gcct_loyaltyprogramcode','gcct_material','gcct_materialrequest','gcct_materialrequestdetail','gcct_offer','gcct_offerdetail','gcct_onlinegaragevehicles','gcct_partsorderstatus','gcct_partsordersystems','gcct_partssmeassistancerequest','gcct_pcarequest','gcct_priordealerdecision','gcct_qaincident','gcct_qamonitoring','gcct_queuepriority','gcct_ravprocessingmilestones','gcct_reactdata','gcct_reacttransmission','gcct_reasoncode','gcct_recall','gcct_region','gcct_rentaldetail','gcct_slaconfiguration','gcct_sms','gcct_socialmediaconversation','gcct_socialmediamessage','gcct_socialmediaprofile','gcct_specialloanercode','gcct_state','gcct_surveyconfiguration','gcct_symptomclassification','gcct_tasktype','gcct_technicalassistancerequest','gcct_testdriverequest','gcct_timezone','gcct_userdelegationofauthority','gcct_vehicle','gcct_vehiclebrand','gcct_vehicleclass','gcct_vehicleloyaltyallowance','gcct_vehicleoffroad','gcct_vehicleowner','gcct_warrantyhistory','gcct_warrantyloanerpartsdelay','gcct_warrantyloanerrequest','gcct_warrantyloanerrequesthistory','globaloptionsetmetadata','incident','incidentresolution','letter','msdyn_answer','msdyn_question','msdyn_questionresponse','msdyn_survey','msdyn_surveyinvite','msdyn_surveyresponse','optionsetmetadata','phonecall','queue','queueitem','sla','slaitem','slakpiinstance','socialactivity','socialprofile','statemetadata','statusmetadata','systemuser','task','team','teammembership','territory','gcct_marketingprogram','gcct_catalogcorrection','gcct_cataloginquiry','gcct_dealerclaims','gcct_fcsinquiry','gcct_partavailabilityinquiry','gcct_partorderstatusinquiry','gcct_partqualitycontrolinquiry','gcct_pricinginquiries','gcct_trafficinquiry','gcct_catalogdeck', 'gcct_partdistributioncenter', 'gcct_portalcasetype']


In [3]:
import xlrd

new=pd.ExcelFile('/s/latluri1/GCCT_EntitySchema_Sep2019.xlsx',encoding=sys.getfilesystemencoding())
#new=pd.ExcelFile('/s/latluri1/Schema_changes/GCCT_EntitySchema_v2.4.xlsx')
#new=pd.ExcelFile('/s/latluri1/DES/GCCT_EntitySchema_v2.5_march21_2019.xlsx')

In [4]:
#take 5 th sheet and create empty data frame
schema_df=new.parse(new.sheet_names[5]).truncate(after=-1)
list_sheets=list(new.sheet_names)
list_sheets.remove('Entities list')
#tables_list=new.parse('Entities list',skiprows=2)['Logical Name']
tables_list=new.parse('Entities list')['Logical Name']


In [5]:

output_name="Sep_2019"
writer = pd.ExcelWriter("GCCT Schema single excel_"+output_name+".xlsx", engine = 'xlsxwriter')

for j,i in enumerate(list_sheets):
    df=new.parse(i)
    df["Table"]=tables_list[j]
    if tables_list[j] in tab_list:
        df["Priority"]="True"
    else:
        df["Priority"]="False"
    schema_df=schema_df.append(df,ignore_index=True,sort=True)
    
    
schema_df.to_excel(writer, sheet_name="Sheet1", index=False)
writer.save()
writer.close()


In [6]:
stmnt='echo \"GCCT schema in single Excel file  '+output_name+'\" | mailx -a GCCT\ Schema\ single\ excel_'+output_name+'.xlsx -s GCCT\ Schema\ single\ file_'+output_name+' latluri1@ford.com'
print(stmnt)

echo "GCCT schema in single Excel file  Sep_2019" | mailx -a GCCT\ Schema\ single\ excel_Sep_2019.xlsx -s GCCT\ Schema\ single\ file_Sep_2019 latluri1@ford.com


In [7]:
commands.getstatusoutput(stmnt)

(0, '')

In [7]:
schema_df

,Additional data,Audit Enabled,Custom Attribute,Description,Display Name,Logical Name,Priority,Required Level,Schema Name,Secured,Table,Type,ValidFor AdvancedFind
0,Options:\n1: Preferred Customer\n2: Standard\n...,True,False,Select a category to indicate whether the cust...,Account Category,accountcategorycode,True,None,AccountCategoryCode,False,account,Picklist,True
1,NaN,False,False,NaN,NaN,accountcategorycodename,True,None,AccountCategoryCodeName,False,account,Virtual,False
2,Options:\n1: Default Value\nDefault: 1,True,False,Select a classification code to indicate the p...,Classification,accountclassificationcode,True,None,AccountClassificationCode,False,account,Picklist,False
3,NaN,False,False,NaN,NaN,accountclassificationcodename,True,None,AccountClassificationCodeName,False,account,Virtual,False
4,NaN,False,False,Unique identifier of the account.,Account,accountid,True,SystemRequired,AccountId,False,account,Uniqueidentifier,True
5,Format: Text\nMax length: 20,True,False,Type an ID number or code for the account to q...,Account Number,accountnumber,True,None,AccountNumber,False,account,String,True
6,Options:\n1: Default Value\nDefault: 1,True,False,Select a rating to indicate the value of the c...,Account Rating,accountratingcode,True,None,AccountRatingCode,False,account,Picklist,False
7,NaN,False,False,NaN,NaN,accountratingcodename,True,None,AccountRatingCodeName,False,account,Virtual,False
8,NaN,False,False,Unique identifier for address 1.,Address 1: ID,address1_addressid,True,None,Address1_AddressId,False,account,Uniqueidentifier,False
9,Options:\n1: Bill To\n2: Ship To\n3: Primary\n...,True,False,Select the primary address type.,Address 1: Address Type,address1_addresstypecode,True,None,Address1_AddressTypeCode,False,account,Picklist,True


In [10]:
informatica_report=pd.read_csv('/s/latluri1/dsc10742_gcctmsd_lz_db_phase3.csv',encoding='ISO-8859-1')
informatica_report

,#DATABASE_NAME,TABLE_NAME,COLUMN_NAME,DATATYPE,INFA_INFERRED_DATATYPE,MINIMUM_VALUE,MAXIMUM_VALUE,DISTINCT_VALUE,DISTINCT_PERCENT,NULL_VALUE_COUNT,...,BLANK_PERCENT,TOTAL_ROWS,TOTAL_SUM,REGEX_PATTERN,SECURITY_LEVEL,LAST_TIME_RUN,APPROVER_CDSID,APPROVED_DATE_TIME,APPROVED_DOMAIN_NAME,APPROVED_SECURITY_LEVEL
0,dsc10742_gcctmsd_lz_db,account,account_identity,bigint,Integer(6) [100.00%],1,685485,78852,22.52,NaN,...,0,350000,3.616137e+10,NaN,NoPII,7/23/2019 2:04,NaN,NaN,NaN,NaN
1,dsc10742_gcctmsd_lz_db,account,account_rowversion,string,Fixed Length String(12) [100.00%],AAAAABf5/Po=,AAAAABpwwVQ=,349991,99.99,NaN,...,0,350000,NaN,NaN,NoPII,7/23/2019 2:04,NaN,NaN,NaN,NaN
2,dsc10742_gcctmsd_lz_db,account,accountcategorycode,integer,NaN,0,0,0,0,NaN,...,0,350000,0.000000e+00,NaN,NoPII,7/23/2019 2:04,NaN,NaN,NaN,NaN
3,dsc10742_gcctmsd_lz_db,account,accountclassificationcode,integer,Integer(1) [100.00%],0,1,1,0.01,NaN,...,0,350000,3.400190e+05,NaN,NoPII,7/23/2019 2:04,NaN,NaN,NaN,NaN
4,dsc10742_gcctmsd_lz_db,account,accountid,string,Fixed Length String(36) [100.00%],00003938-4945-E911-A853-000D3A13AE0A,FFFFA6F6-7A7D-E911-A855-000D3A13A9C6,52297,14.94,NaN,...,0,350000,NaN,Regex_DataDomain_Column_Rule_AccountNumber,SPII,7/23/2019 2:04,NaN,NaN,NaN,NaN
5,dsc10742_gcctmsd_lz_db,account,accountnumber,string,Integer(2) [100.00%],NaN,6,2,0.01,NaN,...,0,350000,NaN,Regex_DataDomain_Column_Rule_AccountNumber,SPII,7/23/2019 2:04,NaN,NaN,NaN,NaN
6,dsc10742_gcctmsd_lz_db,account,accountratingcode,integer,Integer(1) [100.00%],1,1,1,0.01,NaN,...,0,350000,3.400190e+05,NaN,NoPII,7/23/2019 2:04,NaN,NaN,NaN,NaN
7,dsc10742_gcctmsd_lz_db,account,address1_addressid,string,Fixed Length String(36) [100.00%],0000041A-ACEA-489D-A068-22E9D14FA17F,FFFFD6A6-3D3D-43AA-BFD4-4778B0304E1D,78852,22.52,NaN,...,0,350000,NaN,NaN,NoPII,7/23/2019 2:04,NaN,NaN,NaN,NaN
8,dsc10742_gcctmsd_lz_db,account,address1_addresstypecode,integer,Integer(1) [100.00%],1,4,4,0.01,NaN,...,0,350000,2.794000e+03,NaN,NoPII,7/23/2019 2:04,NaN,NaN,NaN,NaN
9,dsc10742_gcctmsd_lz_db,account,address1_city,string,String(31) [100.00%],*,ÃLVARO OBREGÃN,11104,3.17,NaN,...,0,350000,NaN,Regex_DataDomain_Column_Rule_CityOrTown_valida...,PII,7/23/2019 2:04,NaN,NaN,NaN,NaN


In [11]:
final=informatica_report.merge(schema_df,how='left',left_on=['TABLE_NAME','COLUMN_NAME'],right_on=['Table','Logical Name'],suffixes=('_old','_new') )
final=final.drop_duplicates()

In [12]:
pd.value_counts(final.APPROVED_SECURITY_LEVEL,dropna=False).to_frame().reset_index()

,index,APPROVED_SECURITY_LEVEL
0,NaN,11147


In [13]:
global_desc_contains={"PII":[" fin$","^fin$","^vin$"," vin$","weblink","url","sales code","P&A Code","path","title"," fin "," vin ","user","subject","box","fax","financial","Amount","unique","address","telephone","phone","contact","fax","^Type","email"]}

In [14]:
for i in global_desc_contains.keys():
        for j in global_desc_contains[i]:
            final.loc[final["Description"].str.contains(j,na=False,case=False),"APPROVED_SECURITY_LEVEL"]=i

In [15]:
pd.value_counts(final.APPROVED_SECURITY_LEVEL,dropna=False).to_frame().reset_index()

,index,APPROVED_SECURITY_LEVEL
0,NaN,8954
1,PII,2193


In [16]:
global_crm_dtype={"PII":["BigInt","Customer","Double","Lookup","Memo","Money","Owner","PartyList","Picklist","Uniqueidentifier"],"No PII":["Boolean","DateTime","EntityName","Integer","State","Status","Virtual","Decimal"]}

In [17]:
for i in global_crm_dtype.keys():
        for j in global_crm_dtype[i]:
            final.loc[final["Type"]==j,"APPROVED_SECURITY_LEVEL"]=i

In [18]:
pd.value_counts(final.APPROVED_SECURITY_LEVEL,dropna=False).to_frame().reset_index()

,index,APPROVED_SECURITY_LEVEL
0,NaN,5478
1,PII,3208
2,No PII,2461


In [19]:
global_col_name_contains={"PII":["path$","url","security","password","versionnumber","name","zip","post","contact","unique","_identity$","_rowversion$","id$","by$","entitystring","shakey","address","latitude","longitude"],"No PII":["lastupdatedatetime","lastupdtby","eventtype","process__id","message__id","eventdatetime","_entitytype$"]}

In [20]:
for i in global_col_name_contains.keys():
        for j in global_col_name_contains[i]:
            final.loc[final["COLUMN_NAME"].str.contains(j,na=False,case=False),"APPROVED_SECURITY_LEVEL"]=i

In [21]:
pd.value_counts(final.APPROVED_SECURITY_LEVEL,dropna=False).to_frame().reset_index()

,index,APPROVED_SECURITY_LEVEL
0,PII,6240
1,No PII,4307
2,NaN,600


In [28]:
#if the CRM schema is missing=> new fields in GCCT 
#we will have to depend on informatica information
global_informatica_dtpe_contains={"No PII":["integer","date/time"]}

In [29]:
for i in global_informatica_dtpe_contains.keys():
        for j in global_informatica_dtpe_contains[i]:
            final.loc[final["DATATYPE"].str.contains(j,na=False,case=False),"APPROVED_SECURITY_LEVEL"]=i

In [30]:
pd.value_counts(final.APPROVED_SECURITY_LEVEL,dropna=False).to_frame().reset_index()

,index,APPROVED_SECURITY_LEVEL
0,PII,5519
1,No PII,5241
2,NaN,387


In [31]:
final.to_csv("dsc10742_gcctmsd_lz_db_profiling_phase3_v6.csv",index=False,encoding='utf-8')

In [33]:
stmnt='echo \"GCCT profiling workshop phase 3 v6  '+output_name+'\" | mailx -a dsc10742_gcctmsd_lz_db_profiling_phase3_v6.csv -s "gcct profiling workshop phase3 v6" latluri1@ford.com'
print(stmnt)
commands.getstatusoutput(stmnt)

echo "GCCT profiling workshop phase 3 v6  July_2019" | mailx -a dsc10742_gcctmsd_lz_db_profiling_phase3_v6.csv -s "gcct profiling workshop phase3 v6" latluri1@ford.com


(0, '')